In [ ]:
// 1st model of data of aqi ...dataset of all teh 20 cities combined

In [ ]:
//2nd model of aqi 

In [ ]:
//3rd model of aqi

In [ ]:
//4th model of aqi

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
df=pd.read_csv(r'C:\Users\ASUS\Desktop\Climate1\AQI.csv')

In [4]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=["City", "date"])

# Encode city names
city_encoder = LabelEncoder()
df['City_Encoded'] = city_encoder.fit_transform(df['City'])

# Normalize AQI values
scaler = MinMaxScaler()
df['Index Value Scaled'] = scaler.fit_transform(df[['Index Value']])

In [5]:
import seaborn as sns; import matplotlib.pyplot as plt; sns.heatmap(df.corr(), cmap='coolwarm', annot=True); plt.show()


ValueError: could not convert string to float: 'Agartala'

In [2]:
df = pd.read_excel(r'C:\Users\ASUS\Documents\cleaned stage 1 data.xlsx',header=0)

NameError: name 'pd' is not defined

In [21]:


# Load dataset

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=["City", "date"])

# Encode city names
city_encoder = LabelEncoder()
df['City_Encoded'] = city_encoder.fit_transform(df['City'])

# Normalize AQI values
scaler = MinMaxScaler()
df['Index Value Scaled'] = scaler.fit_transform(df[['Index Value']])

# Prepare time-series sequences
SEQ_LENGTH = 180  # 6 months of data
X, y = [], []
cities = df['City_Encoded'].unique()
for city in cities:
    city_data = df[df['City_Encoded'] == city][['Index Value Scaled']].values
    seq_length = min(SEQ_LENGTH, len(city_data))  # Use available data if less than 6 months
    if len(city_data) >= 1:  # Ensure there is at least some data
        X.append(city_data[-seq_length:])
        y.append(city_data[-1])

X, y = np.array(X), np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(None, 1)),
    Dropout(0.2),
    LSTM(50, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Predict function
def predict_aqi(city_name):
    city_encoded = city_encoder.transform([city_name])[0]
    city_data = df[df['City_Encoded'] == city_encoded].sort_values(by='date')
    
    if len(city_data) < 1:
        return "Not enough data for prediction"
    
    seq_length = min(SEQ_LENGTH, len(city_data))
    last_days = city_data['Index Value Scaled'].values[-seq_length:].reshape(1, seq_length, 1)
    prediction = model.predict(last_days)
    return scaler.inverse_transform(prediction)[0][0]

# Example usage
# predicted_aqi = predict_aqi("Delhi")
# print("Predicted AQI:", predicted_aqi)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (30,) + inhomogeneous part.

In [25]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=["City", "date"])

# Encode city names
city_encoder = LabelEncoder()
df['City_Encoded'] = city_encoder.fit_transform(df['City'])

# Normalize AQI values
scaler = MinMaxScaler()
df['Index Value Scaled'] = scaler.fit_transform(df[['Index Value']])

# Prepare time-series sequences
SEQ_LENGTH = 180  # 6 months of data
X, y = [], []
cities = df['City_Encoded'].unique()
for city in cities:
    city_data = df[df['City_Encoded'] == city][['Index Value Scaled']].values
    seq_length = min(SEQ_LENGTH, len(city_data))  # Use available data if less than 6 months
    if len(city_data) >= 1:  # Ensure there is at least some data
        sequence = city_data[-seq_length:].reshape(-1)
        X.append(sequence)
        y.append(city_data[-1])

# Pad sequences to ensure uniform shape
X = pad_sequences(X, maxlen=SEQ_LENGTH, dtype='float32', padding='pre')
y = np.array(y)

X = X.reshape(X.shape[0], X.shape[1], 1)  # Reshape for LSTM input

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(SEQ_LENGTH, 1)),
    Dropout(0.2),
    LSTM(50, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Predict function
def predict_aqi(city_name):
    city_encoded = city_encoder.transform([city_name])[0]
    city_data = df[df['City_Encoded'] == city_encoded].sort_values(by='date')
    
    if len(city_data) < 1:
        return "Not enough data for prediction"
    
    seq_length = min(SEQ_LENGTH, len(city_data))
    last_days = city_data['Index Value Scaled'].values[-seq_length:].reshape(-1)
    last_days = pad_sequences([last_days], maxlen=SEQ_LENGTH, dtype='float32', padding='pre').reshape(1, SEQ_LENGTH, 1)
    
    prediction = model.predict(last_days)
    return scaler.inverse_transform(prediction)[0][0]

d:\Anacondaa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 0.1271 - val_loss: 0.1598
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - loss: 0.1169 - val_loss: 0.1480
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step - loss: 0.1098 - val_loss: 0.1377
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - loss: 0.1018 - val_loss: 0.1282
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step - loss: 0.0973 - val_loss: 0.1194
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - loss: 0.0901 - val_loss: 0.1111
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - loss: 0.0867 - val_loss: 0.1029
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - loss: 0.0768 - val_loss: 0.0945
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - loss: 0.0692 - val_loss: 0.0861
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - loss: 0.0634 - val_loss: 0.0775
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step - loss: 0.0568 - val_loss: 0.0688
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - loss: 0.0524

In [26]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=["City", "date"])

# Encode city names
city_encoder = LabelEncoder()
df['City_Encoded'] = city_encoder.fit_transform(df['City'])

# Normalize AQI values
scaler = MinMaxScaler()
df['Index Value Scaled'] = scaler.fit_transform(df[['Index Value']])

# Prepare time-series sequences
SEQ_LENGTH = 180  # 6 months of data
X, y = [], []
cities = df['City_Encoded'].unique()
for city in cities:
    city_data = df[df['City_Encoded'] == city][['Index Value Scaled']].values
    seq_length = min(SEQ_LENGTH, len(city_data))  # Use available data if less than 6 months
    if len(city_data) >= 1:  # Ensure there is at least some data
        sequence = city_data[-seq_length:].reshape(-1)
        X.append(sequence)
        y.append(city_data[-1])

# Pad sequences to ensure uniform shape
X = pad_sequences(X, maxlen=SEQ_LENGTH, dtype='float32', padding='pre')
y = np.array(y)

X = X.reshape(X.shape[0], X.shape[1], 1)  # Reshape for LSTM input

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(SEQ_LENGTH, 1)),
    Dropout(0.2),
    LSTM(50, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Predict function for future dates
def predict_aqi(city_name, future_date):
    city_encoded = city_encoder.transform([city_name])[0]
    city_data = df[df['City_Encoded'] == city_encoded].sort_values(by='date')
    
    if len(city_data) < 1:
        return "Not enough data for prediction"
    
    seq_length = min(SEQ_LENGTH, len(city_data))
    last_days = city_data['Index Value Scaled'].values[-seq_length:].reshape(-1)
    last_days = pad_sequences([last_days], maxlen=SEQ_LENGTH, dtype='float32', padding='pre').reshape(1, SEQ_LENGTH, 1)
    
    future_date = pd.to_datetime(future_date)
    last_date = city_data['date'].max()
    days_ahead = (future_date - last_date).days
    
    if days_ahead < 0:
        actual_aqi = df[(df['City_Encoded'] == city_encoded) & (df['date'] == future_date)]['Index Value']
        return f"Actual AQI for {city_name} on {future_date.date()}: {actual_aqi.values[0] if not actual_aqi.empty else 'No data available'}"
    
    for _ in range(days_ahead):
        prediction = model.predict(last_days)
        predicted_aqi_scaled = prediction[0][0]
        last_days = np.append(last_days[:, 1:, :], [[[predicted_aqi_scaled]]], axis=1)
    
    predicted_aqi = scaler.inverse_transform([[predicted_aqi_scaled]])[0][0]
    return f"Predicted AQI for {city_name} on {future_date.date()}: {predicted_aqi}"

Epoch 1/20


d:\Anacondaa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 0.1016 - val_loss: 0.1292
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - loss: 0.0955 - val_loss: 0.1217
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - loss: 0.0891 - val_loss: 0.1143
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - loss: 0.0839 - val_loss: 0.1067
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step - loss: 0.0792 - val_loss: 0.0984
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step - loss: 0.0753 - val_loss: 0.0896
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - loss: 0.0652 - val_loss: 0.0805
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - loss: 0.0610 - val_loss: 0.0709
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step - loss: 0.0557 - val_loss: 0.0608
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - loss: 0.0465 - val_loss: 0.0504
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - loss: 0.0414 - val_loss: 0.0397
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - loss: 0.0319 - val_loss: 0.0291
Epo

In [27]:
result = predict_aqi("Delhi", "2026-01-01")
print(result)


ValueError: y contains previously unseen labels: 'Delhi'

In [29]:


df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=["City", "date"])

# Encode city names
city_encoder = LabelEncoder()
df['City_Encoded'] = city_encoder.fit_transform(df['City'])

# Normalize AQI values
scaler = MinMaxScaler()
df['Index Value Scaled'] = scaler.fit_transform(df[['Index Value']])

# Prepare time-series sequences
SEQ_LENGTH = 180  # 6 months of data
X, y = [], []
cities = df['City_Encoded'].unique()
for city in cities:
    city_data = df[df['City_Encoded'] == city][['Index Value Scaled']].values
    seq_length = min(SEQ_LENGTH, len(city_data))  # Use available data if less than 6 months
    if len(city_data) >= 1:  # Ensure there is at least some data
        sequence = city_data[-seq_length:].reshape(-1)
        X.append(sequence)
        y.append(city_data[-1])

# Pad sequences to ensure uniform shape
X = pad_sequences(X, maxlen=SEQ_LENGTH, dtype='float32', padding='pre')
y = np.array(y)

X = X.reshape(X.shape[0], X.shape[1], 1)  # Reshape for LSTM input

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(SEQ_LENGTH, 1)),
    Dropout(0.2),
    LSTM(50, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Predict function for future dates
def predict_aqi(city_name, future_date):
    if city_name not in city_encoder.classes_:
        return f"City '{city_name}' not found in training data"
    
    city_encoded = city_encoder.transform([city_name])[0]
    city_data = df[df['City_Encoded'] == city_encoded].sort_values(by='date')
    
    if len(city_data) < 1:
        return "Not enough data for prediction"
    
    seq_length = min(SEQ_LENGTH, len(city_data))
    last_days = city_data['Index Value Scaled'].values[-seq_length:].reshape(-1)
    last_days = pad_sequences([last_days], maxlen=SEQ_LENGTH, dtype='float32', padding='pre').reshape(1, SEQ_LENGTH, 1)
    
    future_date = pd.to_datetime(future_date)
    last_date = city_data['date'].max()
    days_ahead = (future_date - last_date).days
    
    if days_ahead < 0:
        actual_aqi = df[(df['City_Encoded'] == city_encoded) & (df['date'] == future_date)]['Index Value']
        return f"Actual AQI for {city_name} on {future_date.date()}: {actual_aqi.values[0] if not actual_aqi.empty else 'No data available'}"
    
    for _ in range(days_ahead):
        prediction = model.predict(last_days)
        predicted_aqi_scaled = prediction[0][0]
        last_days = np.append(last_days[:, 1:, :], [[[predicted_aqi_scaled]]], axis=1)
    
    predicted_aqi = scaler.inverse_transform([[predicted_aqi_scaled]])[0][0]
    return f"Predicted AQI for {city_name} on {future_date.date()}: {predicted_aqi}"

Epoch 1/20


d:\Anacondaa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 0.1064 - val_loss: 0.1343
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - loss: 0.0994 - val_loss: 0.1238
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - loss: 0.0931 - val_loss: 0.1139
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - loss: 0.0858 - val_loss: 0.1036
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - loss: 0.0769 - val_loss: 0.0931
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - loss: 0.0678 - val_loss: 0.0823
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - loss: 0.0588 - val_loss: 0.0714
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - loss: 0.0523 - val_loss: 0.0605
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - loss: 0.0461 - val_loss: 0.0490
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - loss: 0.0381 - val_loss: 0.0369
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step - loss: 0.0298 - val_loss: 0.0253
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - loss: 0.0233 - val_loss: 0.0156
Epo

In [31]:
result = predict_aqi("Agra", "2026-01-01")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 582ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━

In [32]:
result = predict_aqi("Agra", "2025-02-02")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━

In [33]:
result = predict_aqi("Agra", "2025-01-01")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━

In [34]:
result = predict_aqi("Agra", "2025-02-25")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
import pickle
from statsmodels.tsa.statespace.sarimax import SARIMAX
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Load dataset
df=pd.read_csv(r'C:\Users\ASUS\Desktop\Climate1\AQI.csv')

df['date'] = pd.to_datetime(df['date'])
df = df.drop_duplicates(subset=['City', 'date'])

# Count unique days per city
city_data_counts = df.groupby('City')['date'].nunique()
cities_with_2_years_data = city_data_counts[city_data_counts > 730].index.tolist()

models = {}
errors = {}

for city in df['City'].unique():
    df_city = df[df['City'] == city].copy()
    df_city.set_index('date', inplace=True)
    df_city = df_city[~df_city.index.duplicated(keep='first')]
    df_city = df_city.asfreq('D')
    
    df_city['Index Value'] = df_city['Index Value'].interpolate(method='linear')
    imputer = IterativeImputer(max_iter=5, random_state=42)
    df_city[['Index Value']] = imputer.fit_transform(df_city[['Index Value']])
    
    # Feature Engineering - Adding Lag Features
    df_city['lag_1'] = df_city['Index Value'].shift(1)
    df_city['lag_7'] = df_city['Index Value'].shift(7)
    df_city['lag_30'] = df_city['Index Value'].shift(30)
    df_city.dropna(inplace=True)
    
    df_city['day_of_year'] = df_city.index.dayofyear
    df_city['year'] = df_city.index.year
    X = df_city[['day_of_year', 'year', 'lag_1', 'lag_7', 'lag_30']]
    y = df_city['Index Value']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    model_xgb = XGBRegressor(objective='reg:squarederror', n_estimators=50, learning_rate=0.05, max_depth=3, n_jobs=-1)
    model_xgb.fit(X_train, y_train)
    y_pred_xgb = model_xgb.predict(X_test)
    error_xgb = mean_absolute_error(y_test, y_pred_xgb)
    
    if city in cities_with_2_years_data:
        y_train_sarima = y_train[-30:]  # Train SARIMAX on last 2 years
        model_sarima = SARIMAX(y_train_sarima, order=(1, 1, 1), seasonal_order=(1, 1, 1, 52), enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
        sarima_pred = model_sarima.forecast(steps=len(y_test))
        error_sarima = mean_absolute_error(y_test, sarima_pred)
        models[city] = (model_xgb, model_sarima)
        errors[city] = (error_xgb, error_sarima)
    else:
        models[city] = (model_xgb, None)
        errors[city] = (error_xgb, None)

with open("aqi_prediction_models.pkl", "wb") as f:
    pickle.dump(models, f)
with open("aqi_prediction_errors.pkl", "wb") as f:
    pickle.dump(errors, f)

def predict_aqi(city, future_date):
    future_date = pd.to_datetime(future_date)
    with open("aqi_prediction_models.pkl", "rb") as f:
        models = pickle.load(f)
    with open("aqi_prediction_errors.pkl", "rb") as f:
        errors = pickle.load(f)
    
    if city not in models:
        return {"Error": "City not found in dataset"}
    
    model_xgb, model_sarima = models[city]
    error_xgb, error_sarima = errors.get(city, (None, None))
    future_features = pd.DataFrame({'day_of_year': [future_date.dayofyear], 'year': [future_date.year], 'lag_1': [np.nan], 'lag_7': [np.nan], 'lag_30': [np.nan]})
    xgb_pred = model_xgb.predict(future_features.dropna(axis=1, how='any'))[0]
    
    if model_sarima:
        sarima_pred = model_sarima.forecast(steps=1)[0]
        final_prediction = 0.5 * xgb_pred + 0.5 * sarima_pred
        confidence = max(0, 100 - ((error_xgb + error_sarima) / 2))
    else:
        final_prediction = xgb_pred
        confidence = max(0, 100 - error_xgb)
    
    return {
        "City": city,
        "Future Date": future_date.strftime('%Y-%m-%d'),
        "Predicted AQI": round(final_prediction, 2),
        "Prediction Confidence": round(confidence, 2)
    }

# Example usage
city_input = input("Enter city name: ")
date_input = input("Enter future date (YYYY-MM-DD): ")
prediction = predict_aqi(city_input, date_input)
print(prediction)


d:\Anacondaa\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
d:\Anacondaa\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
d:\Anacondaa\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
d:\Anacondaa\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
d:\Anacondaa\Lib\site-packages\statsmodels\tsa\ba

ValueError: feature_names mismatch: ['day_of_year', 'year', 'lag_1', 'lag_7', 'lag_30'] ['day_of_year', 'year']
expected lag_7, lag_30, lag_1 in input data